In [36]:
import requests
from bs4 import BeautifulSoup

# Function to fetch submission details from the Codeforces API
def get_submission_details(user_handle, problem_id):
    api_url = f'https://codeforces.com/api/user.status?handle={user_handle}&from=1&count=100'
    print(f"Fetching submissions from: {api_url}")
    
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Codeforces API: {e}")
        return None

    if data['status'] != 'OK':
        print(f"API returned an error: {data.get('comment', 'Unknown error')}")
        return None
    
    

    # Look for the specific problem in the submissions
    for submission in data['result']:
        pID = f"{submission['problem']['contestId']}{submission['problem']['index']}"
        if pID == problem_id:
            print(f"Found matching submission for problem: {pID}")
            print(f"Submission ID: {submission['id']}, Verdict: {submission['verdict']}")
            return submission

    print(f"No submission found for problem ID: {problem_id}")
    return None

# Function to scrape and save the source code from a Codeforces submission page
def scrape_and_save_code(submission_url, submission_id):
    try:
        response = requests.get(submission_url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching submission page: {e}")
        return

    soup = BeautifulSoup(response.content, 'html5lib')
    code_block = soup.find('pre', attrs={'id': 'program-source-text'})

    if code_block:
        code = code_block.text
        print(f"Code fetched successfully for submission {submission_id}")
        
        # Save the code to a file
        with open(f"{submission_id}.txt", 'w') as file:
            file.write(code)
        print(f"Code saved to {submission_id}.txt")
    else:
        print("Could not find the source code on the page.")

# Main function
def main():
    # User inputs
    user_handle = input("Enter your Codeforces handle: ")
    problem_id = input("Enter the problem ID in the format {ContestID + ProblemCode} (e.g., 1999B): ")
    
    # Step 1: Fetch submission details from the API
    submission = get_submission_details(user_handle, problem_id)
    
    if submission:
        # Step 2: Construct the submission URL and scrape the source code
        submission_url = f"https://codeforces.com/contest/{submission['contestId']}/submission/{submission['id']}"
        scrape_and_save_code(submission_url, submission['id'])

if __name__ == "__main__":
    main()


Enter your Codeforces handle:  00bhavikaish
Enter the problem ID in the format {ContestID + ProblemCode} (e.g., 1999B):  1999B


Fetching submissions from: https://codeforces.com/api/user.status?handle=00bhavikaish&from=1&count=100
Found matching submission for problem: 1999B
Submission ID: 274912519, Verdict: WRONG_ANSWER
Error fetching submission page: 403 Client Error: Forbidden for url: https://codeforces.com/contest/1999/submission/274912519
